In [ ]:
from os import path
if not path.exists('./BERTweet/'):
  !git clone https://github.com/VinAIResearch/BERTweet/

In [ ]:
!pip3 install -r /content/requirements.txt
# this file should be uploaded to the content directory in google colab

In [ ]:
import sys
sys.path.append("/content/BERTweet")
from TweetNormalizer import normalizeTweet

In [ ]:
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments
)
import torch
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [ ]:
evaluate_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
seed = 42
dataset = load_dataset("tweet_eval", "emoji")
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/emoji/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def tokenize(data):
    """
    Using the normalize tweet function from bertweet, so the text is optimized
    for the bertweet model.
    Then the text is tokenized and padded and truncated. The max length is set
    by the max length of the bertweet model.
    """
    return tokenizer(normalizeTweet(data["text"]), padding="max_length", truncation=True)


def compute_metrics(p):
    """
    This will be run after each epoch. It will calculate the accuracy, macro
    recall, macro precision and macro f1-score.
    """
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average="macro")
    precision = precision_score(y_true=labels, y_pred=pred, average="macro")
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}



In [ ]:
dataset = dataset.map(tokenize)
train = dataset["train"].shuffle(seed=45)
eval = dataset["validation"].shuffle(seed=45)
test = dataset["test"].shuffle(seed=45)

print(train[0:5])

Map:   0%|          | 0/45000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

{'text': ['GLK x Busdriver @ Low End Thoery', '#haircut #alert #thanks Kevin, I ️it!!! @ Andy Lecompte Salon', '@user was on us.. Literally. #whenitsdarkout @ Jimmy Kimmel Live!', "It's Lit! #BeatThegiants #FirstDodgerGameofTheSeason #ITFDB #GoDodgers #iLoveLA…", "Diva's 9th birthday party. We sang to HER. @ Crystal Palace…"], 'label': [4, 0, 2, 4, 5], 'input_ids': [[0, 12361, 794, 314, 13442, 3202, 5, 4393, 1744, 16577, 3474, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 85, 9199, 8927, 14610, 30892, 3638, 7, 8, 3, 18, 12, 12, 12, 5, 5505, 20673, 1504, 581, 1336, 16076, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
model = AutoModelForSequenceClassification \
    .from_pretrained("vinai/bertweet-base", num_labels=20)

training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    learning_rate=0.00005,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    weight_decay=0,
    num_train_epochs=10
)

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: 

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=compute_metrics,
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 45000
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 7040
  Number of trainable parameters = 134915348


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.311000,2.067162,0.373400,0.285023,0.255992,0.232309
2,1.975500,2.010669,0.395200,0.345031,0.295348,0.264051
3,1.634500,2.069947,0.390200,0.325006,0.298064,0.287925
4,1.431400,2.211628,0.383200,0.344266,0.312363,0.300696
5,1.134700,2.352373,0.374600,0.352381,0.328000,0.327138
6,0.922200,2.559505,0.368600,0.333708,0.321202,0.315849
7,0.813300,2.734706,0.359200,0.336181,0.318701,0.319770
8,0.595500,2.881815,0.356600,0.325449,0.316119,0.318219
9,0.528200,3.029438,0.359800,0.333694,0.316916,0.320615
10,0.421600,3.073787,0.358200,0.328094,0.319087,0.321470


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 64
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_

TrainOutput(global_step=7040, training_loss=1.1440170897678896, metrics={'train_runtime': 8978.5511, 'train_samples_per_second': 50.119, 'train_steps_per_second': 0.784, 'total_flos': 2.96047775232e+16, 'train_loss': 1.1440170897678896, 'epoch': 10.0})

In [ ]:
model.eval()
x = trainer.predict(test)
x

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 50000
  Batch size = 64


PredictionOutput(predictions=array([[ 0.5151153 ,  1.6520486 ,  0.75824106, ..., -0.4336553 ,
         0.6463456 , -0.6883605 ],
       [ 1.5526067 ,  7.6458893 , -0.09961555, ..., -1.8971908 ,
        -0.9492333 , -1.3336141 ],
       [ 2.7177808 ,  5.5907826 , -1.3208126 , ..., -2.9157422 ,
        -2.362361  , -2.6719315 ],
       ...,
       [ 1.1923735 , -0.08286557, -2.4448116 , ..., -0.5884552 ,
        -1.72111   , -0.6013299 ],
       [-1.4722898 , -2.5236099 , -0.469313  , ...,  0.22653076,
         0.7207315 ,  0.78251415],
       [ 0.25310168,  3.811935  ,  4.890359  , ..., -2.1029203 ,
        -1.3742354 ,  0.51616305]], dtype=float32), label_ids=array([ 2,  0, 13, ...,  1,  7,  2]), metrics={'test_loss': 3.1435978412628174, 'test_accuracy': 0.34614, 'test_precision': 0.2972857589065593, 'test_recall': 0.2927621743991965, 'test_f1': 0.2916816385579718, 'test_runtime': 323.3832, 'test_samples_per_second': 154.615, 'test_steps_per_second': 2.418})